In [1]:
import json
import pdfplumber
import spacy
import pandas as pd
import numpy as np
import os
import unidecode
import random
from spacy.matcher import PhraseMatcher
from spacy import displacy
from progress_bar.progress_bar import printProgressBar
from pdfminer.high_level import extract_text

In [2]:
#using gpu
spacy.prefer_gpu()

True

In [3]:
#getting all the files
df = pd.read_csv('data/url_thesis_8211_exist.csv')


In [4]:
# Filtering by only files that exist in the folder.
df_result = df[df['exist'] == True]
df_result

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True
1,2,2010,La Economía del Ocio como una alternativa de e...,"Rivera, Elizabeth",2 MB,http://saber.ucv.ve/handle/10872/16245,http://saber.ucv.ve/bitstream/10872/16245/1/TG...,NaN,thesis_pdf_all/2.pdf,True
2,3,27-May-2011,La producción de conocimiento : un análisis mé...,"Adrian Mogollón, Julia; Echezuria Perozo, Jorg...",995.92 kB,http://saber.ucv.ve/handle/10872/29,http://saber.ucv.ve/bitstream/10872/29/1/TL-EB...,NaN,thesis_pdf_all/3.pdf,True
3,4,27-May-2011,Desarrollo de un algoritmo computacional para ...,"D'Amico, Walter F.",506.32 kB,http://saber.ucv.ve/handle/10872/32,http://saber.ucv.ve/bitstream/10872/32/1/Tesis...,NaN,thesis_pdf_all/4.pdf,True
4,5,27-May-2011,Evaluación geofísica integral de los conos alu...,"Cragno, Analia V.; Romero, Marcos A.",3.02 MB,http://saber.ucv.ve/handle/10872/31,http://saber.ucv.ve/bitstream/10872/31/1/Evalu...,NaN,thesis_pdf_all/5.pdf,True
...,...,...,...,...,...,...,...,...,...,...
8206,8207,4-Dec-2020,Elaboración de un manual de procedimientos e i...,"Armas R., Rodolfo J.",924.37 kB,http://saber.ucv.ve/handle/10872/20906,http://saber.ucv.ve/bitstream/10872/20906/1/TE...,NaN,thesis_pdf_all/8207.pdf,True
8207,8208,8-Mar-2021,Condiciones de vida de los estudiantes de Geoq...,"Pérez, Andrea; Pinto, Hendrith",7.21 MB,http://saber.ucv.ve/handle/10872/20969,http://saber.ucv.ve/bitstream/10872/20969/1/CO...,NaN,thesis_pdf_all/8208.pdf,True
8208,8209,8-Mar-2021,Análisis de los elementos pospornográficos de ...,"Belandria, Sofía",1.13 MB,http://saber.ucv.ve/handle/10872/20970,http://saber.ucv.ve/bitstream/10872/20970/1/So...,NaN,thesis_pdf_all/8209.pdf,True
8209,8210,11-Mar-2021,La Fecundidad Adolescente en Venezuela vista a...,"Simao, Edith Carolina",3.56 MB,http://saber.ucv.ve/handle/10872/20973,http://saber.ucv.ve/bitstream/10872/20973/1/TF...,NaN,thesis_pdf_all/8210.pdf,True


In [5]:
l = len(df_result.index)
l

8173

In [6]:
def is_image_n_page(file, n_page = 1):
    """ Determinates if a especific
    page of a file has text or is scanned.

    Parameters:
    -----------
    file : str
        The file location of the pdf file.
    n_page: int, optional
        The page number to check, must be less the the maximum ammoun of page of the pdf. 
        By default it checks on the first page.
    
    Returns:
    -----------
    boolean
        A boolean value if the page has text returns FALSE, if it's a image it return TRUE.
    """
    with pdfplumber.open(file) as pdf:
        if n_page < len(pdf.pages) and n_page >= 0:
            page = pdf.pages[n_page]
            #print(n_page)
        else:
            return(True)
        #print(len(pdf.pages))
        #print(random.randrange(2,len(pdf.pages)))

        text = page.extract_text()
        #print(text)  
    if text == '':
        return(True)
    else:
        return(False)


def check_conditions(file, thesis_title, l, conditions_to_meet = 4):
    """Checks if the whole pdf file is an image or has enough text to be treated as text.
    It work with 5 conditions described in parameters.

    Parameters:
    -----------
    file : str
        The file location of the pdf file.
    conditions_to_meet: int, optional
        The number of conditions to meet, it goes with first page, second page,
        the middle page plus one, the page -20 from the end, and a random page that is not the first and the second.
        By defualt it check the conditions in order and it must meet at least 4 to be True.
    
    Returns:
    -----------
    boolean
        A boolean value that if you meet the number of 
        conditions_to_meet parameter then it return True.
    """
    printProgressBar(int(thesis_title), l, prefix = 'Progress:', suffix = 'Complete', length = 50)
    conditions = [False, False, False, False, False]
    print(file)
    try:
        if is_image_n_page(file, 0):
            conditions[0] = True
        if is_image_n_page(file, 1):
            conditions[1] = True
        with pdfplumber.open(file) as pdf:
            if is_image_n_page(file, int(len(pdf.pages)/2) + 1):
                conditions[2] = True
            if is_image_n_page(file, len(pdf.pages)-20):
                conditions[3] = True
            if is_image_n_page(file, random.randrange(2,len(pdf.pages))):
                conditions[4] = True
        if int(sum(conditions)) >= conditions_to_meet:
            return True
        else:
            return False
    except:
        return None

In [7]:
# try:
#     with pdfplumber.open('../thesis_pdf_all/358.pdf') as pdf:
#         page = pdf.pages[0]
#         text = page.extract_text()
#     print(text)
# except:
#     print("yo wtf?")

In [8]:
def isPDF(file, index, l):
    printProgressBar(int(index), l, prefix = 'Progress:', suffix = 'Complete', length = 50)
    #check whether it is a pdf file or no
    try:
        with pdfplumber.open(file) as pdf:
            page = pdf.pages[0]
            text = page.extract_text()
        return True
    except:
        return False

In [9]:
l = len(df_result.index)
df_result = df_result.copy()
func_isPDF = np.vectorize(isPDF)
values_pdf = func_isPDF("../" + df_result['path'], df_result['index'], l)
df_result['isPDF'] = values_pdf.tolist()

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [10]:
df_result['isPDF'] = values_pdf.tolist()

In [11]:
df_result

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True
1,2,2010,La Economía del Ocio como una alternativa de e...,"Rivera, Elizabeth",2 MB,http://saber.ucv.ve/handle/10872/16245,http://saber.ucv.ve/bitstream/10872/16245/1/TG...,NaN,thesis_pdf_all/2.pdf,True,True
2,3,27-May-2011,La producción de conocimiento : un análisis mé...,"Adrian Mogollón, Julia; Echezuria Perozo, Jorg...",995.92 kB,http://saber.ucv.ve/handle/10872/29,http://saber.ucv.ve/bitstream/10872/29/1/TL-EB...,NaN,thesis_pdf_all/3.pdf,True,True
3,4,27-May-2011,Desarrollo de un algoritmo computacional para ...,"D'Amico, Walter F.",506.32 kB,http://saber.ucv.ve/handle/10872/32,http://saber.ucv.ve/bitstream/10872/32/1/Tesis...,NaN,thesis_pdf_all/4.pdf,True,True
4,5,27-May-2011,Evaluación geofísica integral de los conos alu...,"Cragno, Analia V.; Romero, Marcos A.",3.02 MB,http://saber.ucv.ve/handle/10872/31,http://saber.ucv.ve/bitstream/10872/31/1/Evalu...,NaN,thesis_pdf_all/5.pdf,True,True
...,...,...,...,...,...,...,...,...,...,...,...
8206,8207,4-Dec-2020,Elaboración de un manual de procedimientos e i...,"Armas R., Rodolfo J.",924.37 kB,http://saber.ucv.ve/handle/10872/20906,http://saber.ucv.ve/bitstream/10872/20906/1/TE...,NaN,thesis_pdf_all/8207.pdf,True,True
8207,8208,8-Mar-2021,Condiciones de vida de los estudiantes de Geoq...,"Pérez, Andrea; Pinto, Hendrith",7.21 MB,http://saber.ucv.ve/handle/10872/20969,http://saber.ucv.ve/bitstream/10872/20969/1/CO...,NaN,thesis_pdf_all/8208.pdf,True,True
8208,8209,8-Mar-2021,Análisis de los elementos pospornográficos de ...,"Belandria, Sofía",1.13 MB,http://saber.ucv.ve/handle/10872/20970,http://saber.ucv.ve/bitstream/10872/20970/1/So...,NaN,thesis_pdf_all/8209.pdf,True,True
8209,8210,11-Mar-2021,La Fecundidad Adolescente en Venezuela vista a...,"Simao, Edith Carolina",3.56 MB,http://saber.ucv.ve/handle/10872/20973,http://saber.ucv.ve/bitstream/10872/20973/1/TF...,NaN,thesis_pdf_all/8210.pdf,True,True


In [12]:
csv_dest = 'data/url_thesis_8211_with_pdf_scan.csv'

In [13]:
df_result.to_csv(csv_dest, index=False)

In [14]:
new_df = df_result.copy()
new_df

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True
1,2,2010,La Economía del Ocio como una alternativa de e...,"Rivera, Elizabeth",2 MB,http://saber.ucv.ve/handle/10872/16245,http://saber.ucv.ve/bitstream/10872/16245/1/TG...,NaN,thesis_pdf_all/2.pdf,True,True
2,3,27-May-2011,La producción de conocimiento : un análisis mé...,"Adrian Mogollón, Julia; Echezuria Perozo, Jorg...",995.92 kB,http://saber.ucv.ve/handle/10872/29,http://saber.ucv.ve/bitstream/10872/29/1/TL-EB...,NaN,thesis_pdf_all/3.pdf,True,True
3,4,27-May-2011,Desarrollo de un algoritmo computacional para ...,"D'Amico, Walter F.",506.32 kB,http://saber.ucv.ve/handle/10872/32,http://saber.ucv.ve/bitstream/10872/32/1/Tesis...,NaN,thesis_pdf_all/4.pdf,True,True
4,5,27-May-2011,Evaluación geofísica integral de los conos alu...,"Cragno, Analia V.; Romero, Marcos A.",3.02 MB,http://saber.ucv.ve/handle/10872/31,http://saber.ucv.ve/bitstream/10872/31/1/Evalu...,NaN,thesis_pdf_all/5.pdf,True,True
...,...,...,...,...,...,...,...,...,...,...,...
8206,8207,4-Dec-2020,Elaboración de un manual de procedimientos e i...,"Armas R., Rodolfo J.",924.37 kB,http://saber.ucv.ve/handle/10872/20906,http://saber.ucv.ve/bitstream/10872/20906/1/TE...,NaN,thesis_pdf_all/8207.pdf,True,True
8207,8208,8-Mar-2021,Condiciones de vida de los estudiantes de Geoq...,"Pérez, Andrea; Pinto, Hendrith",7.21 MB,http://saber.ucv.ve/handle/10872/20969,http://saber.ucv.ve/bitstream/10872/20969/1/CO...,NaN,thesis_pdf_all/8208.pdf,True,True
8208,8209,8-Mar-2021,Análisis de los elementos pospornográficos de ...,"Belandria, Sofía",1.13 MB,http://saber.ucv.ve/handle/10872/20970,http://saber.ucv.ve/bitstream/10872/20970/1/So...,NaN,thesis_pdf_all/8209.pdf,True,True
8209,8210,11-Mar-2021,La Fecundidad Adolescente en Venezuela vista a...,"Simao, Edith Carolina",3.56 MB,http://saber.ucv.ve/handle/10872/20973,http://saber.ucv.ve/bitstream/10872/20973/1/TF...,NaN,thesis_pdf_all/8210.pdf,True,True


In [15]:
new_df = new_df[new_df['isPDF']==True]
new_df

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True
1,2,2010,La Economía del Ocio como una alternativa de e...,"Rivera, Elizabeth",2 MB,http://saber.ucv.ve/handle/10872/16245,http://saber.ucv.ve/bitstream/10872/16245/1/TG...,NaN,thesis_pdf_all/2.pdf,True,True
2,3,27-May-2011,La producción de conocimiento : un análisis mé...,"Adrian Mogollón, Julia; Echezuria Perozo, Jorg...",995.92 kB,http://saber.ucv.ve/handle/10872/29,http://saber.ucv.ve/bitstream/10872/29/1/TL-EB...,NaN,thesis_pdf_all/3.pdf,True,True
3,4,27-May-2011,Desarrollo de un algoritmo computacional para ...,"D'Amico, Walter F.",506.32 kB,http://saber.ucv.ve/handle/10872/32,http://saber.ucv.ve/bitstream/10872/32/1/Tesis...,NaN,thesis_pdf_all/4.pdf,True,True
4,5,27-May-2011,Evaluación geofísica integral de los conos alu...,"Cragno, Analia V.; Romero, Marcos A.",3.02 MB,http://saber.ucv.ve/handle/10872/31,http://saber.ucv.ve/bitstream/10872/31/1/Evalu...,NaN,thesis_pdf_all/5.pdf,True,True
...,...,...,...,...,...,...,...,...,...,...,...
8206,8207,4-Dec-2020,Elaboración de un manual de procedimientos e i...,"Armas R., Rodolfo J.",924.37 kB,http://saber.ucv.ve/handle/10872/20906,http://saber.ucv.ve/bitstream/10872/20906/1/TE...,NaN,thesis_pdf_all/8207.pdf,True,True
8207,8208,8-Mar-2021,Condiciones de vida de los estudiantes de Geoq...,"Pérez, Andrea; Pinto, Hendrith",7.21 MB,http://saber.ucv.ve/handle/10872/20969,http://saber.ucv.ve/bitstream/10872/20969/1/CO...,NaN,thesis_pdf_all/8208.pdf,True,True
8208,8209,8-Mar-2021,Análisis de los elementos pospornográficos de ...,"Belandria, Sofía",1.13 MB,http://saber.ucv.ve/handle/10872/20970,http://saber.ucv.ve/bitstream/10872/20970/1/So...,NaN,thesis_pdf_all/8209.pdf,True,True
8209,8210,11-Mar-2021,La Fecundidad Adolescente en Venezuela vista a...,"Simao, Edith Carolina",3.56 MB,http://saber.ucv.ve/handle/10872/20973,http://saber.ucv.ve/bitstream/10872/20973/1/TF...,NaN,thesis_pdf_all/8210.pdf,True,True


In [16]:
csv_dest = 'data/url_thesis_8211_with_pdf_scan_image.csv'

In [17]:
df_result_scan = pd.read_csv('data/url_thesis_8211_with_pdf_scan.csv')
df_result_scan = df_result_scan[df_result_scan['isPDF']==True]
df_result_scan

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True
1,2,2010,La Economía del Ocio como una alternativa de e...,"Rivera, Elizabeth",2 MB,http://saber.ucv.ve/handle/10872/16245,http://saber.ucv.ve/bitstream/10872/16245/1/TG...,NaN,thesis_pdf_all/2.pdf,True,True
2,3,27-May-2011,La producción de conocimiento : un análisis mé...,"Adrian Mogollón, Julia; Echezuria Perozo, Jorg...",995.92 kB,http://saber.ucv.ve/handle/10872/29,http://saber.ucv.ve/bitstream/10872/29/1/TL-EB...,NaN,thesis_pdf_all/3.pdf,True,True
3,4,27-May-2011,Desarrollo de un algoritmo computacional para ...,"D'Amico, Walter F.",506.32 kB,http://saber.ucv.ve/handle/10872/32,http://saber.ucv.ve/bitstream/10872/32/1/Tesis...,NaN,thesis_pdf_all/4.pdf,True,True
4,5,27-May-2011,Evaluación geofísica integral de los conos alu...,"Cragno, Analia V.; Romero, Marcos A.",3.02 MB,http://saber.ucv.ve/handle/10872/31,http://saber.ucv.ve/bitstream/10872/31/1/Evalu...,NaN,thesis_pdf_all/5.pdf,True,True
...,...,...,...,...,...,...,...,...,...,...,...
8168,8207,4-Dec-2020,Elaboración de un manual de procedimientos e i...,"Armas R., Rodolfo J.",924.37 kB,http://saber.ucv.ve/handle/10872/20906,http://saber.ucv.ve/bitstream/10872/20906/1/TE...,NaN,thesis_pdf_all/8207.pdf,True,True
8169,8208,8-Mar-2021,Condiciones de vida de los estudiantes de Geoq...,"Pérez, Andrea; Pinto, Hendrith",7.21 MB,http://saber.ucv.ve/handle/10872/20969,http://saber.ucv.ve/bitstream/10872/20969/1/CO...,NaN,thesis_pdf_all/8208.pdf,True,True
8170,8209,8-Mar-2021,Análisis de los elementos pospornográficos de ...,"Belandria, Sofía",1.13 MB,http://saber.ucv.ve/handle/10872/20970,http://saber.ucv.ve/bitstream/10872/20970/1/So...,NaN,thesis_pdf_all/8209.pdf,True,True
8171,8210,11-Mar-2021,La Fecundidad Adolescente en Venezuela vista a...,"Simao, Edith Carolina",3.56 MB,http://saber.ucv.ve/handle/10872/20973,http://saber.ucv.ve/bitstream/10872/20973/1/TF...,NaN,thesis_pdf_all/8210.pdf,True,True


In [18]:
csv_dest = 'data/url_thesis_8211_with_pdf_scan_image.csv'

In [19]:

l = len(df_result_scan.index)
#df_head['path'] = source_folder
#func_concat = np.vectorize(concatenate)
#values = func_concat(df_head['path'], df_head['index'])
#df_head['path'] = values.tolist()

func_check = np.vectorize(check_conditions)
values_c = func_check("../" + df_result_scan['path'], df_result_scan['index'], l)

df_result_scan['isScan'] = values_c.tolist()
df_result_scan.to_csv(csv_dest, index=False)

../thesis_pdf_all/1.pdf--------------------------------------| 0.0% Complete
../thesis_pdf_all/1.pdf--------------------------------------| 0.0% Complete
../thesis_pdf_all/2.pdf--------------------------------------| 0.0% Complete
../thesis_pdf_all/3.pdf--------------------------------------| 0.0% Complete
../thesis_pdf_all/4.pdf--------------------------------------| 0.1% Complete
../thesis_pdf_all/5.pdf--------------------------------------| 0.1% Complete
../thesis_pdf_all/6.pdf--------------------------------------| 0.1% Complete
../thesis_pdf_all/7.pdf--------------------------------------| 0.1% Complete
../thesis_pdf_all/8.pdf--------------------------------------| 0.1% Complete
../thesis_pdf_all/9.pdf--------------------------------------| 0.1% Complete
../thesis_pdf_all/10.pdf-------------------------------------| 0.1% Complete
../thesis_pdf_all/11.pdf-------------------------------------| 0.1% Complete
../thesis_pdf_all/12.pdf-------------------------------------| 0.2% Complete

../thesis_pdf_all/107.pdf------------------------------------| 1.4% Complete
../thesis_pdf_all/108.pdf------------------------------------| 1.4% Complete
../thesis_pdf_all/109.pdf------------------------------------| 1.4% Complete
../thesis_pdf_all/110.pdf------------------------------------| 1.4% Complete
../thesis_pdf_all/111.pdf------------------------------------| 1.4% Complete
../thesis_pdf_all/112.pdf------------------------------------| 1.4% Complete
../thesis_pdf_all/113.pdf------------------------------------| 1.4% Complete
../thesis_pdf_all/114.pdf------------------------------------| 1.4% Complete
../thesis_pdf_all/115.pdf------------------------------------| 1.5% Complete
../thesis_pdf_all/116.pdf------------------------------------| 1.5% Complete
../thesis_pdf_all/117.pdf------------------------------------| 1.5% Complete
../thesis_pdf_all/118.pdf------------------------------------| 1.5% Complete
../thesis_pdf_all/119.pdf------------------------------------| 1.5% Complete

../thesis_pdf_all/215.pdf------------------------------------| 2.7% Complete
../thesis_pdf_all/216.pdf------------------------------------| 2.7% Complete
../thesis_pdf_all/217.pdf------------------------------------| 2.8% Complete
../thesis_pdf_all/218.pdf------------------------------------| 2.8% Complete
../thesis_pdf_all/219.pdf------------------------------------| 2.8% Complete
../thesis_pdf_all/220.pdf------------------------------------| 2.8% Complete
../thesis_pdf_all/221.pdf------------------------------------| 2.8% Complete
../thesis_pdf_all/222.pdf------------------------------------| 2.8% Complete
../thesis_pdf_all/223.pdf------------------------------------| 2.8% Complete
../thesis_pdf_all/224.pdf------------------------------------| 2.8% Complete
../thesis_pdf_all/225.pdf------------------------------------| 2.9% Complete
../thesis_pdf_all/226.pdf------------------------------------| 2.9% Complete
../thesis_pdf_all/227.pdf------------------------------------| 2.9% Complete

../thesis_pdf_all/322.pdf------------------------------------| 4.1% Complete
../thesis_pdf_all/323.pdf------------------------------------| 4.1% Complete
../thesis_pdf_all/324.pdf------------------------------------| 4.1% Complete
../thesis_pdf_all/325.pdf------------------------------------| 4.1% Complete
../thesis_pdf_all/326.pdf------------------------------------| 4.1% Complete
../thesis_pdf_all/327.pdf------------------------------------| 4.1% Complete
../thesis_pdf_all/328.pdf------------------------------------| 4.2% Complete
../thesis_pdf_all/329.pdf------------------------------------| 4.2% Complete
../thesis_pdf_all/330.pdf------------------------------------| 4.2% Complete
../thesis_pdf_all/331.pdf------------------------------------| 4.2% Complete
../thesis_pdf_all/332.pdf------------------------------------| 4.2% Complete
../thesis_pdf_all/333.pdf------------------------------------| 4.2% Complete
../thesis_pdf_all/334.pdf------------------------------------| 4.2% Complete

../thesis_pdf_all/648.pdf------------------------------------| 8.2% Complete
../thesis_pdf_all/649.pdf------------------------------------| 8.2% Complete
../thesis_pdf_all/650.pdf------------------------------------| 8.2% Complete
../thesis_pdf_all/651.pdf------------------------------------| 8.3% Complete
../thesis_pdf_all/652.pdf------------------------------------| 8.3% Complete
../thesis_pdf_all/653.pdf------------------------------------| 8.3% Complete
../thesis_pdf_all/654.pdf------------------------------------| 8.3% Complete
../thesis_pdf_all/655.pdf------------------------------------| 8.3% Complete
../thesis_pdf_all/656.pdf------------------------------------| 8.3% Complete
../thesis_pdf_all/657.pdf------------------------------------| 8.3% Complete
../thesis_pdf_all/658.pdf------------------------------------| 8.3% Complete
../thesis_pdf_all/659.pdf------------------------------------| 8.4% Complete
../thesis_pdf_all/660.pdf------------------------------------| 8.4% Complete

...

../thesis_pdf_all/7749.pdf██████████████████████████████████-| 98.3% Complete
../thesis_pdf_all/7750.pdf██████████████████████████████████-| 98.3% Complete
../thesis_pdf_all/7751.pdf██████████████████████████████████-| 98.3% Complete
../thesis_pdf_all/7752.pdf██████████████████████████████████-| 98.3% Complete
../thesis_pdf_all/7753.pdf██████████████████████████████████-| 98.4% Complete
../thesis_pdf_all/7754.pdf██████████████████████████████████-| 98.4% Complete
../thesis_pdf_all/7755.pdf██████████████████████████████████-| 98.4% Complete
../thesis_pdf_all/7756.pdf██████████████████████████████████-| 98.4% Complete
../thesis_pdf_all/7757.pdf██████████████████████████████████-| 98.4% Complete
../thesis_pdf_all/7758.pdf██████████████████████████████████-| 98.4% Complete
../thesis_pdf_all/7759.pdf██████████████████████████████████-| 98.4% Complete
../thesis_pdf_all/7760.pdf██████████████████████████████████-| 98.4% Complete
../thesis_pdf_all/7761.pdf██████████████████████████████████-| 9

../thesis_pdf_all/7862.pdf██████████████████████████████████-| 99.7% Complete
../thesis_pdf_all/7863.pdf██████████████████████████████████-| 99.7% Complete
../thesis_pdf_all/7864.pdf██████████████████████████████████-| 99.8% Complete
../thesis_pdf_all/7865.pdf██████████████████████████████████-| 99.8% Complete
../thesis_pdf_all/7866.pdf██████████████████████████████████-| 99.8% Complete
../thesis_pdf_all/7867.pdf██████████████████████████████████-| 99.8% Complete
../thesis_pdf_all/7868.pdf██████████████████████████████████-| 99.8% Complete
../thesis_pdf_all/7869.pdf██████████████████████████████████-| 99.8% Complete
../thesis_pdf_all/7870.pdf██████████████████████████████████-| 99.8% Complete
../thesis_pdf_all/7871.pdf██████████████████████████████████-| 99.8% Complete
../thesis_pdf_all/7872.pdf██████████████████████████████████-| 99.9% Complete
../thesis_pdf_all/7873.pdf██████████████████████████████████-| 99.9% Complete
../thesis_pdf_all/7874.pdf██████████████████████████████████-| 9

../thesis_pdf_all/7969.pdf███████████████████████████████████| 101.1% Complete
../thesis_pdf_all/7970.pdf███████████████████████████████████| 101.1% Complete
../thesis_pdf_all/7971.pdf███████████████████████████████████| 101.1% Complete
../thesis_pdf_all/7972.pdf███████████████████████████████████| 101.1% Complete
../thesis_pdf_all/7973.pdf███████████████████████████████████| 101.1% Complete
../thesis_pdf_all/7974.pdf███████████████████████████████████| 101.2% Complete
../thesis_pdf_all/7975.pdf███████████████████████████████████| 101.2% Complete
../thesis_pdf_all/7976.pdf███████████████████████████████████| 101.2% Complete
../thesis_pdf_all/7977.pdf███████████████████████████████████| 101.2% Complete
../thesis_pdf_all/7978.pdf███████████████████████████████████| 101.2% Complete
../thesis_pdf_all/7979.pdf███████████████████████████████████| 101.2% Complete
../thesis_pdf_all/7980.pdf███████████████████████████████████| 101.2% Complete
../thesis_pdf_all/7981.pdf██████████████████████████

../thesis_pdf_all/8074.pdf████████████████████████████████████| 102.4% Complete
../thesis_pdf_all/8075.pdf████████████████████████████████████| 102.4% Complete
../thesis_pdf_all/8076.pdf████████████████████████████████████| 102.4% Complete
../thesis_pdf_all/8077.pdf████████████████████████████████████| 102.5% Complete
../thesis_pdf_all/8078.pdf████████████████████████████████████| 102.5% Complete
../thesis_pdf_all/8079.pdf████████████████████████████████████| 102.5% Complete
../thesis_pdf_all/8080.pdf████████████████████████████████████| 102.5% Complete
../thesis_pdf_all/8081.pdf████████████████████████████████████| 102.5% Complete
../thesis_pdf_all/8082.pdf████████████████████████████████████| 102.5% Complete
../thesis_pdf_all/8083.pdf████████████████████████████████████| 102.5% Complete
../thesis_pdf_all/8084.pdf████████████████████████████████████| 102.5% Complete
../thesis_pdf_all/8085.pdf████████████████████████████████████| 102.6% Complete
../thesis_pdf_all/8086.pdf██████████████

../thesis_pdf_all/8177.pdf████████████████████████████████████| 103.7% Complete
../thesis_pdf_all/8178.pdf████████████████████████████████████| 103.7% Complete
../thesis_pdf_all/8179.pdf████████████████████████████████████| 103.8% Complete
../thesis_pdf_all/8180.pdf████████████████████████████████████| 103.8% Complete
../thesis_pdf_all/8181.pdf████████████████████████████████████| 103.8% Complete
../thesis_pdf_all/8182.pdf████████████████████████████████████| 103.8% Complete
../thesis_pdf_all/8183.pdf████████████████████████████████████| 103.8% Complete
../thesis_pdf_all/8184.pdf████████████████████████████████████| 103.8% Complete
../thesis_pdf_all/8185.pdf████████████████████████████████████| 103.8% Complete
../thesis_pdf_all/8186.pdf████████████████████████████████████| 103.8% Complete
../thesis_pdf_all/8187.pdf████████████████████████████████████| 103.9% Complete
../thesis_pdf_all/8188.pdf████████████████████████████████████| 103.9% Complete
../thesis_pdf_all/8189.pdf██████████████